## S4g example notebook
Example notebook of the S4g 4 channel, 18-bit current source module. To use this, we need a S4g module and a controller module. This can be either the C1b/C2 combination or the C1.

To use the D5a module, we also need to import the SPI rack class. All communication with the SPI Rack will go through this object. Only one SPI_rack object can be active at a time on the PC. So before running another script, the connection in this one should be closed.

In [1]:
# Import SPI rack and S4g module
from spirack import SPI_rack, S4g_module

Open the SPI rack connection and unlock the controller. This is necessary after bootup of the controller module. If not unlocked, no communication with the modules can take place. The virtual COM port baud rate is irrelevant as it doesn't change the actual speed. Timeout can be changed, but 1 second is a good value.

In [3]:
COM_speed = 1e6 # Baud rate, doesn't matter much
timeout = 1 # In seconds

spi_rack = SPI_rack('COM4', COM_speed, timeout)
spi_rack.unlock() # Unlock the controller to be able to send data

Create a new S4g module object at the correct (set) module address using the SPI object. By default the module resets the output currents to 0 Volt. Before it does this, it will read back the current value. If this value is non-zero it will slowly ramp it to zero. If `reset_currents = False` then the output will not be changed.

Most S4g units have a span of &pm;50mA. If this is not the case, the maximum span can be set at initialisation using the `max_current` parameter. In this case we set it to 50 mA.

In [15]:
S4g = S4g_module(spi_rack, module=2, max_current=50e-3, reset_currents=True)

The output span of the module can be set in software to the multiple ranges. In all these ranges the units keeps the 18-bit resolution. The actual current output depends on the maximum current. Here we asume the default 50mA:
* 0 to 50mA: range_max_uni
* -50 to 50 mA: range_max_bi
* -25 to 25 mA: range_min_bi

By default it is set to &pm;50mA. Changing the span determines the smallest step size possible. The DACs used internally are 18-bit, which gives a smallest step size of:

$$\text{LSB} = \frac{\text{Span}}{2^{18}}$$

In the case of a &pm;25mA span this gives a LSB of: ~190 nA. You can get the current stepsize of a certain current output with the `get_stepsize` function.

In [20]:
S4g.change_span_update(0, S4g.range_min_bi)

stepsize =  S4g.get_stepsize(1)
print("Stepsize: " + str(stepsize/1e-9) + " nA")

Stepsize: 19.073486328125 nA


The current can be set using the `set_current` function. If you want the output to be precisely equal to the set value, the current should be an integer multiple of the stepsize. Especially if sweeps are performed this is recommended. Otherwise the steps might nog be equidistant. Here we set the current of output 0 to 1 mA. In the software the output count starts at 0, while on the front it starts at 1.

Optionally the output current might also be changed by setting a digital bit value ranging from 0 to $2^{18}-1$ using `change_value_update`.

In [32]:
# Changing the output by current
S4g.set_current(0, 1e-3)

In [25]:
# Changing the output by ditigal values
S4g.change_value_update(0, 165535)

When done with the measurement, it is recommended to close the SPI Rack connection. This will allow other measurement scripts to acces the device.

In [22]:
spi_rack.close()